In [1]:
import init
from uuoskit import uuosapi
from uuoskit import wasmcompiler
init.init()
test_account1 = init.test_account1

In [2]:
uuosapi.get_account(test_account1)

{'account_name': 'helloworld11',
 'head_block_num': 5611987,
 'head_block_time': '2021-08-21T14:41:15.000',
 'privileged': False,
 'last_code_update': '2021-08-21T14:34:06.000',
 'created': '2021-02-07T17:59:33.000',
 'core_liquid_balance': '100001004.3384 EOS',
 'ram_quota': 15730017,
 'net_weight': '83716500642',
 'cpu_weight': '334875932576',
 'net_limit': {'used': 4576,
  'available': '27480972565094',
  'max': '27480972569670',
  'last_usage_update_time': {'slot': 227623885},
  'current_used': 4554},
 'cpu_limit': {'used': 8793,
  'available': '39306727818232',
  'max': '39306727827025',
  'last_usage_update_time': {'slot': 227623885},
  'current_used': 8750},
 'ram_usage': 299556,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV',
      'weight': 1}],
    'accounts': [{'permission': {'actor': 'helloworld11',
       'permission': 'eosio.code'},
      'weig

In [3]:
a = {
    "account": test_account1,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":test_account1,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = uuosapi.push_action('eosio', 'updateauth', a, {test_account1:'active'})
print('done!')

done!


In [4]:
test_account1

'helloworld11'

In [6]:
code = r'''
package main

import "chain"
import _ "chain/logger"

type Transfer struct {
    From     chain.Name
    To       chain.Name
    Quantity chain.Asset
    Memo     string
}

func main() {
    _, _, action := chain.GetApplyArgs()
    if action == chain.NewName("sayhello") {
        a := chain.NewAction(chain.NewName("eosio.token"), chain.NewName("transfer"))
        a.AddPermission(chain.NewName("helloworld11"), chain.ActiveName)

        t := Transfer{}
        t.From = chain.NewName("helloworld11")
        t.To = chain.NewName("eosio")
        
        // Send 1.0 EOS
        t.Quantity.Amount = 10000;
        t.Quantity.Symbol = chain.NewSymbol("EOS", 4);

        a.Data = t.Pack()
        a.Send()
    }
    
}
'''
code, abi = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [7]:
uuosapi.get_balance(test_account1), uuosapi.get_balance('eosio')

(100001004.3384, 589987464.7273)

In [8]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])


1778


In [9]:
uuosapi.get_balance(test_account1), uuosapi.get_balance('eosio')

(100001003.3384, 589987465.7273)